In [1]:
import pandas as pd
import numpy as np
from catnip.fla_redshift import FLA_Redshift
from sqlalchemy import null
from datetime import datetime

from prefect.blocks.system import Secret
from typing import Dict
from concurrent.futures import ThreadPoolExecutor

In [2]:
def get_redshift_credentials() -> Dict:

    cred_dict = {
        "dbname": Secret.load("stellar-redshift-db-name").get(),
        "host": Secret.load("stellar-redshift-host").get(),
        "port": 5439,
        "user": Secret.load("stellar-redshift-user-name").get(),
        "password": Secret.load("stellar-redshift-password").get(),

        "aws_access_key_id": Secret.load("fla-s3-aws-access-key-id-east-1").get(),
        "aws_secret_access_key": Secret.load("fla-s3-aws-secret-access-key-east-1").get(),
        "bucket": Secret.load("fla-s3-bucket-name-east-1").get(),
        "subdirectory": "us-east-1",

        "verbose": False,
    }

    return cred_dict
    
with ThreadPoolExecutor(1) as pool:
    rs_creds = pool.submit(lambda: get_redshift_credentials()).result()

In [ ]:
q = """
WITH initial AS (
    SELECT
        lower(primary_email) as email,
        MIN(fan_create_date) AS fan_create_date
    FROM
        custom.blade_pii
    WHERE
        fan_crt_orgn_name ilike '%future_goals%'
        OR fan_crt_orgn_name ilike '%everfi%'
    GROUP BY
        email
),
acct_rev_info AS (
    SELECT
        contacts.email,
        creation_date AS add_date,
        CASE
            WHEN ticket.season LIKE '2021-22' THEN gross_revenue
            ELSE 0
        END AS "gross_rev_2122",
        CASE
            WHEN ticket.season LIKE '2022-23' THEN gross_revenue
            ELSE 0
        END AS "gross_rev_2223",
        CASE
            WHEN ticket.season LIKE '2023-24' THEN gross_revenue
            ELSE 0
        END AS "gross_rev_2324",
        CASE
            WHEN ticket.season LIKE '2024-25' THEN gross_revenue
            ELSE 0
        END AS "gross_rev_2425",
        CASE
            WHEN ticket.season LIKE '2021-22' AND "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2122",
        CASE
            WHEN ticket.season LIKE '2022-23' AND "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2223",
        CASE
            WHEN ticket.season LIKE '2023-24' AND "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2324",
        CASE
            WHEN ticket.season LIKE '2024-25' AND "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2425"
    FROM
        custom.cth_v_historical_ticket ticket
    INNER JOIN
        custom.korepss_v_contacts contacts ON ticket.purchaser_ticketing_id = contacts.primary_ticketing_id
),
subs_2526 as (
    SELECT
        email,
        sum(gross_revenue) AS "gross_rev_2526",
        1 AS "is_stm_2526"
    FROM
        custom.cth_v_historical_plans plans
    INNER JOIN
        custom.korepss_v_contacts contacts ON plans.purchaser_ticketing_id = contacts.primary_ticketing_id
    GROUP BY
        email
),
acct_rev_agg AS (
    SELECT
        email,
        MIN(add_date) AS add_date,
        SUM(gross_rev_2122) AS gross_rev_2122,
        SUM(gross_rev_2223) AS gross_rev_2223,
        SUM(gross_rev_2324) AS gross_rev_2324,
        SUM(gross_rev_2425) AS gross_rev_2425,
        MAX(is_stm_2122) AS is_stm_2122,
        MAX(is_stm_2223) AS is_stm_2223,
        MAX(is_stm_2324) AS is_stm_2324,
        MAX(is_stm_2425) AS is_stm_2425
    FROM
        acct_rev_info
    GROUP BY
        email
)
SELECT DISTINCT
    initial.email,
    fan_create_date::date,
    add_date,
    CASE
        WHEN add_date < fan_create_date THEN 1
        ELSE 0
    END AS pre_blade_email,
    CASE
        WHEN add_date > fan_create_date THEN 1
        ELSE 0
    END AS post_blade_email,
    CASE
        WHEN add_date IS NULL THEN 1
        ELSE 0
    END AS no_tickets,
    COALESCE(gross_rev_2122, 0) AS gross_rev_2122,
    COALESCE(gross_rev_2223, 0) AS gross_rev_2223,
    COALESCE(gross_rev_2324, 0) AS gross_rev_2324,
    COALESCE(gross_rev_2425, 0) AS gross_rev_2425,
    COALESCE(gross_rev_2526, 0) AS gross_rev_2526,
    COALESCE(is_stm_2122, 0) AS is_stm_2122,
    COALESCE(is_stm_2223, 0) AS is_stm_2223,
    COALESCE(is_stm_2324, 0) AS is_stm_2324,
    COALESCE(is_stm_2425, 0) AS is_stm_2425,
    COALESCE(is_stm_2526, 0) AS is_stm_2526
FROM
    initial
LEFT JOIN
    acct_rev_agg ON initial.email = acct_rev_agg.email
LEFT JOIN
    subs_2526 ON initial.email = subs_2526.email
WHERE
    initial.email IS NOT NULL
    AND initial.email <> ''
"""

df = FLA_Redshift(**rs_creds).query_warehouse(sql_string = q)

In [35]:
def get_season(date):
    if date.month < 7:
        start_year = date.year - 1
        end_year = date.year % 100
    else:
        start_year = date.year
        end_year = (date.year + 1) % 100
    return f"{start_year}-{end_year:02d}"

# Apply the function to create the 'season' column
df['season'] = df['fan_create_date'].apply(get_season)

In [39]:
season_dict = {
    "2021-22" : 1,
    "2022-23" : 2,
    "2023-24" : 3,
    "2024-25" : 4,
    "2025-26" : 5
}

# sum revenue before/after
for key, value in season_dict.items():
    col_list = list(df.columns[6:(6+value)])
    # print(col_list)
    df.loc[(df['season'] == key), 'pre_blade_revenue'] =  df[col_list].sum(axis = 1)

for key, value in season_dict.items():
    col_list = list(df.columns[(6 + value):11])
    # print(col_list)
    df.loc[(df['season'] == key), 'post_blade_revenue'] =  df[col_list].sum(axis = 1)

# sum stms before/after
for key, value in season_dict.items():
    col_list = list(df.columns[11:(11+value)])
    # print(col_list)
    df.loc[(df['season'] == key), 'pre_blade_tickets'] =  df[col_list].sum(axis = 1)


for key, value in season_dict.items():
    col_list = list(df.columns[(11 + value):16])
    # print(col_list)
    df.loc[(df['season'] == key), 'post_blade_tickets'] =  df[col_list].sum(axis = 1)